In [10]:
# -*- coding: utf-8 -*-
import sys
import os
import bs4
import json
import urllib
import requests
import re
#from da_vinci import Image
from time import sleep


# クロールしたいユーザのID
crawle_user_id = 'nnahito'


def outputCsv(screen_name, user_name, tweet_timestamp, tweet_body):
    """
    # データをCSVに書き出す
    #  @params screen_name       string              ユーザのスクリーン上の名前（例：Nな人）
    #  @params user_name         string              ユーザの半角英数の名前（例：nnahito）
    #  @params tweet_timestamp   string              ツイート日時
    #  @params tweet_body        string              ツイート本文
    """

    # 書き出すファイル名を決定（ユーザ名.csv）
    file_name = 'outoput.csv'

    # 改行などを削除
    screen_name = screen_name.replace('\n','').replace('\r','')
    user_name = user_name.replace('\n','').replace('\r','')
    tweet_body = tweet_body.replace('\n','').replace('\r','')

    # 出力
    f = open(file_name, 'a')
    f.write(screen_name + ',' + user_name + ',' + tweet_timestamp + ',' + tweet_body + ',' + "\n")
    f.close()


def parseHtml(html_data):
    """
    # 与えられたHTMLをパースしてファイルに書き出します
    #  @paramas html_data           解析したいHTML
    """
    # HTMLを解析
    soup = bs4.BeautifulSoup(html_data)

    # ツイートをブロック単位（ul）に分解
    tweet_blocks = soup.select('.tweet')

    # ツイートのブロックを一つずつ抽出
    for tweet_block in tweet_blocks:
        # スクリーンネームを取得
        screen_name = tweet_block.select('.fullname')[0].text

        # ユーザネームを取得
        user_name = tweet_block.select('.username')[0].text

        # ツイートの日時を取得
        tweet_timestamp = tweet_block.select('.tweet-timestamp')[0].text

        # ツイート内容を取得
        tweet_body = tweet_block.select('.tweet-text')[0].text

        # CSVに出力
        print(screen_name + ',' + user_name + ',' + tweet_timestamp + ',' + tweet_body + ',' + "\n")
        outputCsv(screen_name, user_name, tweet_timestamp, tweet_body)


def getMinPosition(html_data):
    """
    # 次の取得開始位置を抽出し、返します
    #  @params html_data        string          解析するHTMLデータを渡します
    #  @return                  string          次の開始位置（min_position）が返ります
    """
    # HTMLを解析
    soup = bs4.BeautifulSoup(html_data)

    # min-positionを抽出
    min_position = soup.select('div[data-min-position]')[0].attrs['data-min-position']

    # 値を返却
    return min_position


def getNextTweet(user_id, max_position):
    """
    # 与えられたポジションから再帰的にクロールを始めます
    #  @params user_id      string          Twitter上の半角英数のユーザ名（例：nnahito）
    #  @params max_position string          次のTwitterのつぶやきID
    """
    # URLの作成
    base_url = 'https://twitter.com/i/profiles/show/'+ user_id +'/timeline/tweets?include_available_features=1&include_entities=1&max_position='+ max_position +'&reset_error_state=false'

    # ベースURLからデータを取得
    url_data = urllib.request.urlopen(base_url)

    # JSON形式でデータを取得
    content = json.loads(url_data.read().decode('utf8'))

    # 次取得のIDを記録しておく
    next_position = content['min_position']

    # HTMLをパースしてファイルに書き出し
    parseHtml(content['items_html'])
    print(next_position + 'の出力完了')

    # 一応スリープをかませる
    sleep(1)

    # 再起的に呼び出す
    getNextTweet(user_id, next_position)


def getFirstTweet(user_id):
    """
    # 与えられたUserIDから、最初のTweetを取得します
    #  @params user_id      string          取得したいユーザの半角英数のユーザ名（例：nnahito）
    """
    # ベースURLからデータを取得
    url_data = urllib.request.urlopen('https://twitter.com/' + user_id)

    # HTMLデータを取得
    html_data = url_data.read()

    # パースして書き出し
    parseHtml(html_data)

    # 取得開始位置を取得
    max_position = getMinPosition(html_data)

    # 次
    getNextTweet(crawle_user_id, max_position)

# プログラム実行
getFirstTweet(crawle_user_id)


/home/shuichi/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/shuichi/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Nな人,@nnahito,6 時間6 時間前,teratailのバッジ、やっと100個集まった。pic.twitter.com/zics0lyqF5,

Nな人,@nnahito,15 時間15 時間前,AI-CONの動画でてるーhttps://www.youtube.com/watch?v=TuUUS4GgV8o …,

Nな人,@nnahito,17 時間17 時間前,I'm home!,

湊川 🌱わかばちゃんと学ぶGA3/27発売,@llminatoll,24 時間24 時間前,代々木公園付近のかた、気をつけて！https://twitter.com/EtsushiNakano/status/971297936792604672 …,

Nな人,@nnahito,3月7日,to ぬけた,

Nな人,@nnahito,3月7日,Franz is down now...
It cant connect online account...,

Nな人,@nnahito,3月7日,Franzオンラインアカウントに繋がらないぞ…？,

Nな人,@nnahito,3月7日,わかるそれな,

Nな人,@nnahito,3月7日,ねむし！！！！！,

Nな人,@nnahito,3月6日,https://nnahito.com  もそろそろタグ検索できるようにしないとなぁ……
実験的にタグの自動割り振りとかさせたい感ある。,

Nな人,@nnahito,3月6日,AI-CON（ https://ai-con.lawyer/  ）の動画CMかっこいいなー！
公開が楽しみ,

 👽すぺくたぁ 👽＠JS裏垢,@Sp4cG00dR1ch,3月5日,PCパーツを買う人と買わない人の特徴

買わない人
「性能ガー」
「値段ガー」
「時期ガー」
「中身ガー」

買う人
「気絶してたら手に持ってた」
「手が滑った」
「床から生えてきた」
「なにこれ」
「クリアファイル買ったら無料で付いてきた」,

魚類,@AME_ZARI,3月4日,自分をリオレウスだと思いこんでいるマグロpic.twitter.com/auDlu3bdpf,

Nな人,@nnahito,3月6日,Dockerの原因解明。
ESETだったｗ,

Nな人,@nnahito,3月6日,誰かボクのDockerを助けて,

山本俊,

Nな人,@nnahito,1月30日,と言いつつ、グリザイアを……,

Nな人,@nnahito,1月30日,おはようございます。
仕事行かねば,

Nな人,@nnahito,1月29日,ねる,

Nな人,@nnahito,1月29日,える,

Nな人,@nnahito,1月29日,おしろかった,

Nな人,@nnahito,1月29日,pic.twitter.com/teo3fMHvPN,

Nな人,@nnahito,1月29日,はぅあああああpic.twitter.com/2d6kcq5eRm,

Nな人,@nnahito,1月29日,ああ、これねれないやつですね。
寝れないやつですわpic.twitter.com/NDr3ztR5tx,

Nな人,@nnahito,1月29日,痩せないと……,

Nな人,@nnahito,1月29日,お腹すくね、この時間,

Nな人,@nnahito,1月29日,結局,

Nな人,@nnahito,1月29日,相変わらず、桐谷華さんの声は癒やしてくるpic.twitter.com/K8hUT1JZ1w,

Nな人,@nnahito,1月29日,リファレンストーンpic.twitter.com/kMGIWvXDFM,

Nな人,@nnahito,1月28日,（ポプテピピック）,

Nな人,@nnahito,1月28日,何気ないLGTMが、すべてのサービスを停止させた,

Nな人,@nnahito,1月28日,ふひっpic.twitter.com/frjziPX6Dz,

Nな人,@nnahito,1月28日,これこれ！こういうのですよ！！
俺が求めてるものは！！！！！pic.twitter.com/lO9BZ6T9Xi,

Nな人,@nnahito,1月28日,やめろ……やめろ…………pic.twitter.com/HXVkHdvYiu,

はちゃち ⛩️バーチャル神エナガYouTuber,@hatyati,2017年4月18日,やったー！！
Live2DサーバルちゃんをUnityで動かしてそれをWallpaper Engineに読み込ませたら動く壁紙にできたー！！pic.twitter.com/YYqTBWWp26,

Nな人,@nnahito,1月28日,誰か、京都から紅華持ってきて,

957459604627075072

Nな人,@nnahito,2017年12月18日,【Nな人のWebしるべ】ビットコインをクラウドでマイニングしてみる https://nnahito.com/articles/17  #Nな人のWebしるべ,

モーセル(全角),@mxsxrxx,2017年12月17日,#2017年自分が選ぶ今年の4枚 #Live2Dpic.twitter.com/EyQNv5j1tA,

Nな人,@nnahito,2017年12月17日,困ったなぁ,

Nな人,@nnahito,2017年12月17日,目の腫れが引かない,

Nな人,@nnahito,2017年12月17日,目に薬塗ってた,

Nな人,@nnahito,2017年12月17日,【Nな人のWebしるべ】ビットコインをクラウドでマイニングしてみる https://nnahito.com/articles/17  #nitiasa,

Nな人,@nnahito,2017年12月17日,ギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたいギター弾きたい,

Nな人,@nnahito,2017年12月17日,入門用でいいからエレキほしい……
もうダメ、マジギター弾きたい……,

Nな人,@nnahito,2017年12月17日,こういうバンドしたいhttps://www.youtube.com/watch?v=-c4XewcGbc0 …,

Nな人,@nnahito,2017年12月17日,勉強会で知り合った人と友だちになり、1/19にセッションすることになったので会社にギター持っていく,

ちゃんおぎ,@muripo_life,2017年12月16日,こんなものが…

Introducing: Unity Machine Learning Agents https://blogs.unity3d.com/2017/09/19/introducing-unity-machine-learning-agents/ … @unity3dさんから,

Miku Nozaki,@mikupaccho,2017年12月16

KeyboardInterrupt: 